## Note
This notebook is used to genrate and save outputs as pickle files 

In [1]:
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
from typing import Dict, List, Optional

import pickle
import numpy as np
import pandas as pd
import arviz as az
import plotly.express as px
import plotly.graph_objects as go

import estival.priors as esp
import estival.targets as est
from estival.model import BayesianCompartmentalModel
from estival.sampling import tools as esamp

from tbdynamics.constants import QUANTILES
from tbdynamics.settings import DATA_PATH, BASE_PATH, VN_PATH
from tbdynamics.tools.inputs import load_params, load_targets, matrix
from tbdynamics.vietnam.model import build_model
from tbdynamics.vietnam.constants import params_name
from tbdynamics.vietnam.calibration.utils import (
    calculate_scenario_outputs,
    calculate_covid_diff_cum_quantiles,
    calculate_scenario_diff_cum_quantiles,
    calculate_diff_cum_detection_reduction,
    get_targets,
    get_all_priors,
    get_bcm
)
from tbdynamics.calibration.plotting import plot_abs_diff_scatter_multi
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from typing import List, Optional, Literal

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
# extract data - only run for first time
RUN_PATH = Path.cwd().parent.parent / 'runs/results'
OUT_PATH = Path.cwd().parent.parent / 'data/outputs/vietnam/pickle'

In [6]:
idata_raw = az.from_netcdf(RUN_PATH / 'calib_full_out_detection_and_contact.nc')
burnt_idata = idata_raw.sel(draw=np.s_[5000:])
idata = az.extract(burnt_idata, num_samples=500)
# inference_data = az.convert_to_inference_data(idata_extract.reset_index('sample'))
# az.to_netcdf(inference_data, OUT_PATH /'extracted_data.nc')

In [7]:
idata

<xarray.Dataset> Size: 84kB
Dimensions:                                         (sample: 500)
Coordinates:
  * sample                                          (sample) object 4kB MultiIndex
  * chain                                           (sample) int32 2kB 4 3 ... 2
  * draw                                            (sample) int32 2kB 8960 ....
Data variables: (12/19)
    contact_rate                                    (sample) float64 4kB 0.25...
    rr_infection_latent                             (sample) float64 4kB 0.01...
    rr_infection_recovered                          (sample) float64 4kB 0.15...
    progression_multiplier                          (sample) float64 4kB 0.63...
    incidence_props_smear_positive_among_pulmonary  (sample) float64 4kB 0.22...
    incidence_props_pulmonary                       (sample) float64 4kB 0.55...
    ...                                              ...
    contact_reduction                               (sample) float64 4kB 0.64...
    detection_reduction                             (sample) float64 4kB 0.80...
    notif_dispersion                                (sample) float64 4kB 0.28...
    prev_dispersion                                 (sample) float64 4kB 50.4...
    sptb_dispersion                                 (sample) float64 4kB 29.1...
    ptb_dispersion                                  (sample) float64 4kB 8.30...
Attributes:
    created_at:                 2025-05-13T17:31:47.207119
    arviz_version:              0.17.0
    inference_library:          pymc
    inference_library_version:  5.2.0
    sampling_time:              1904.5396823883057
    tuning_steps:               5000

In [8]:
#  Load saved idata
# idata = az.from_netcdf(OUT_PATH / 'idata/idata_detection_and_contact.nc')
params = {
    "start_population_size": 2000000.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0,
}
scenario_config = {"detection_reduction": True, "contact_reduction": True}

In [ ]:
# bcm = get_bcm(params, scenario_config, None)
# base_results = esamp.model_results_for_samples(idata, bcm).results

### Calculate the basecase outputs with scenarios of improving case detection

In [ ]:
# basecase = calculate_scenario_outputs(params, idata)
# with open(OUT_PATH / 'quant_outputs3.pkl', 'wb') as f:
#      pickle.dump(basecase, f)

### Output for differences in cumulative diseased and cumulative deaths of COVID-19 vs no COVID-19

In [ ]:
covid_cum_outputs = calculate_covid_diff_cum_quantiles(params, idata)
with open(OUT_PATH / 'covid_diff_quantiles_sens2.pkl', 'wb') as f:
     pickle.dump(covid_cum_outputs, f)

### TB notifications in with different settings of COVID-19, with log likelihood

In [ ]:
# notif_covid_outputs = calculate_notifications_for_covid(params, idata)
# with open(OUT_PATH /'notif_for_covid_with_ll.pkl', 'wb') as f:
#      pickle.dump(notif_covid_outputs, f)

### Cumulative diseased and death with different case detection scenarios

In [ ]:
# scenarios_diff_quantiles = calculate_scenario_diff_cum_quantiles(params, idata, [2.0, 5.0, 12.0], extreme_transmission=True)
# with open(OUT_PATH/ 'scenarios_diff_outputs2.pkl', 'wb') as f:
#     pickle.dump(scenarios_diff_quantiles,f)